In [42]:
# Importing the required libraries

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler

from sklearn.model_selection import train_test_split, RandomizedSearchCV
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt
import seaborn as sns

import os
import warnings
warnings.filterwarnings('ignore')


In [8]:
train_data = pd.read_csv("/content/drive/MyDrive/dl_practice/hp_train.csv", index_col=0)

In [ ]:
train_data.head()

In [10]:
train_data.shape

(1460, 80)

In [11]:
# Make sure you dont drop y
train_data.dropna(axis=1, inplace=True)

In [12]:
train_data.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,2,2008,WD,Normal,208500
2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,0,0,0,0,0,5,2007,WD,Normal,181500
3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,9,2008,WD,Normal,223500
4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [13]:
train_data.columns

Index(['MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageCars', 'GarageArea', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition', 'SalePrice'],
      dtype='object')

In [20]:
train_data.corr()[-1:].columns

Index(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [32]:
X = train_data[
    [
      'LotArea',
     'LotConfig', 
      'YrSold',
      'OverallQual',
      'YearBuilt',
     'TotalBsmtSF'
  ]
]
y = train_data.SalePrice

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [76]:
X.dtypes

LotArea         int64
LotConfig      object
YrSold          int64
OverallQual     int64
YearBuilt       int64
TotalBsmtSF     int64
dtype: object

In [77]:
X.columns

Index(['LotArea', 'LotConfig', 'YrSold', 'OverallQual', 'YearBuilt',
       'TotalBsmtSF'],
      dtype='object')

In [78]:
nomimal_variables = [
  'LotConfig'
]
num_cols = ['LotArea', 'YrSold', 'OverallQual', 'YearBuilt',
       'TotalBsmtSF']

scaler = MinMaxScaler()
nominal_encoder = OneHotEncoder(handle_unknown="ignore")


preprocessor = ColumnTransformer(
    transformers=[
        #("nom_encode", nominal_encoder, nomimal_variables),
        ("scale", scaler, num_cols)
    ]
)


In [79]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [106]:
model = Sequential()
#model.add(Dense(512, kernel_initializer='normal', activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

In [107]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='val_loss', patience=15)

In [108]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_absolute_error')

In [111]:
history = model.fit(
    X_train,
    y_train,
    epochs=500,
    batch_size=512,
    validation_data=(X_test, y_test),
    callbacks=[callback])

Epoch 1/500
2/2 [==============================] - 0s 51ms/step - loss: 30658.3359 - val_loss: 26271.4766
Epoch 2/500
2/2 [==============================] - 0s 31ms/step - loss: 30747.7852 - val_loss: 26246.8633
Epoch 3/500
2/2 [==============================] - 0s 31ms/step - loss: 30409.4863 - val_loss: 26188.6465
Epoch 4/500
2/2 [==============================] - 0s 29ms/step - loss: 31250.5391 - val_loss: 26180.4883
Epoch 5/500
2/2 [==============================] - 0s 31ms/step - loss: 30694.0918 - val_loss: 26226.6113
Epoch 6/500
2/2 [==============================] - 0s 27ms/step - loss: 31436.2422 - val_loss: 26170.3867
Epoch 7/500
2/2 [==============================] - 0s 29ms/step - loss: 31286.2715 - val_loss: 26158.1406
Epoch 8/500
2/2 [==============================] - 0s 28ms/step - loss: 31036.0312 - val_loss: 26202.8516
Epoch 9/500
2/2 [==============================] - 0s 33ms/step - loss: 31400.1797 - val_loss: 26139.0000
Epoch 10/500
2/2 [============================

In [112]:
y_pred = model.predict(X_test)


14/14 [==============================] - 0s 2ms/step


In [113]:
from sklearn.metrics import r2_score

r2_score(y_pred, y_test)

0.5596440611693168